<a href="https://colab.research.google.com/github/FelipePinheiro964/PUCRS---Fase-2---Steam-Games/blob/main/PUCRS_Fase_2_Steam_games.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print(np.version)

<module 'numpy.version' from '/usr/local/lib/python3.10/dist-packages/numpy/version.py'>


*TRABALHO DE PROGRAMAÇÃO PARA DADOS*

_NOME:_ Felipe Pinheiro Fossá -
_CURSO:_ Banco de Dados: ênfase em data Analytics

In [37]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

#seleção de arquivo
f = input('Digite o nome do arquivo: ')

#finalização forçada
if str.lower(f) == 'sair':
  print('Saindo...')
  sys.exit('O sistema foi finalizado pelo usuario, inicie novamente para outro uso.')
else:
  tabela = pd.read_csv(f, sep=",")
  if tabela.empty:
    print('Arquivo vazio.')

match input('Digite o número da questão: '):
  case '0':
    print('Saindo...')
    sys.exit('O sistema foi finalizado pelo usuario, inicie novamente para outro uso.')
  case '1': #questão 1
    try:

      questao_1 = tabela[['Name', 'Metacritic score', 'Release date']]

      if questao_1['Release date'].duplicated().any():
        questao_1_order = questao_1.sort_values(by= 'Release date' and 'Metacritic score', ascending=True)# Mostra os 10 maiores
        display(questao_1_order.tail(n=10))                                                               #false mostrava os 10 menores

      else:

        questao_1_10 = questao_1.copy()
        top_10_notas = questao_1_10.nlargest(10, 'Metacritic score').sort_values(by='Metacritic score', ascending=False) # mostra os maiores por ter poucos dados
        display(top_10_notas.tail(n=10))

    except FileNotFoundError:
      print('Digite um nome válido')

  case '2':
        #try:
          mask = tabela['Genres'].str.contains('RPG', case=False, na=False)
          display(mask[:23])



Digite o nome do arquivo: steam_games.csv
Digite o número da questão: 2


,Genres
0,False
1,False
2,False
3,False
4,False
5,True
6,False
7,False
8,True
9,False
